[Jump to weekly activity](#Weekly-activity)

# Learning outcomes
1. Overview of CV
2. Revisit some important concepts of images as Numpy array.
3. Cropping. Why? One of the data augmentation techniques in deep learning model development.
4. Splitting and merging of color channels
5. Mathematical operations
6. Image blending (add 2 images together)

## Setup (import modules)

In [ ]:
#!pip install opencv-contrib-python

In [ ]:
import sys
assert sys.version_info >= (3, 7)

import numpy as np
import cv2 as cv
from util_func import show_img

## Images as Numpy array

In [ ]:
img = np.zeros((2, 4), dtype=np.uint8)
print(img)

The above 'img' variable belongs to grayscale image. Another primary types of image is **color image**

In [ ]:
img_color = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
print(img_color)

In [ ]:
img[0, 1] = 50
img[1, 2] = 100
print(img)

In [ ]:
img_color1 = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
print(img_color1)

## Access elements in an array

In [ ]:
img = cv.imread("images/lena.jfif")

a = img[49, 199, 2]
b = img.item(49, 199, 2)
a == b

In [ ]:
%timeit a = img[49, 199, 2]
%timeit b = img.item(49, 199, 2)

## Numpy array slicing

In [ ]:
# Extract the top left region of lena image
show_img("lena", img)

In [ ]:
h, w = img.shape[:2]

topleft = img[:h//2, :w//2]

show_img("topleft", topleft)

In [ ]:
# extract central region of the image
yc, wc = h//2, w//2

centre = img[yc-30:yc+30, wc-30: wc+30]

show_img("centre", centre)

## Exercise
1. Create a 200 x 200 white image and display it.
2. Leverage your image processing skills to create a simple wallpaper design as shown in the following image: ![wallpaper](image_embed/exercise_w4.jpg)

Exercise 1

In [ ]:
#create white image
white = np.zeros((200, 200)) + 255
white = np.uint8(white)

show_img("white", white)

Exercise 2

In [ ]:
# generate a repeating pattern
img_arr = np.zeros((30, 30), dtype=np.uint8)

img_arr[:10, 10:20] = 255
img_arr[10:20, :10] = 255
img_arr[10:20, 20:] = 255
img_arr[20:, 10:20] = 255

img = np.tile(img_arr, (3,3))

show_img("pattern", img)

In [ ]:
# extract flower
img = cv.imread("images/flower.jfif")

show_img("flower", img)

In [ ]:
[i for i in dir(cv) if i.startswith('EVENT')]

Extract the region of interest (flower) from the 'flower.jfif'.

In [ ]:
# method 1: callback function
img = cv.imread("images/flower.jfif")
img_copy = img.copy()

def rect_region(event, x, y, flags, params):
    """This is mouseclick callback function"""
    if event == cv.EVENT_LBUTTONDOWN:
        print((x, y))
        cv.circle(img, (x, y), 1, (0, 0, 255), -1)
        cv.imshow("img", img)
        
cv.imshow("img", img)
cv.setMouseCallback("img", rect_region)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
flower = img_copy[39:122, 93:175]

show_img("flower", flower)

In [ ]:
# method 2
img = cv.imread("images/flower.jfif")
bbox = cv.selectROI("img", img)

# (x, y, w, h)
flower = img[int(bbox[1]):  int(bbox[1]+bbox[3]),
            int(bbox[0]): int(bbox[0]+bbox[2])]

show_img("flower", flower)

In [ ]:
# method 3: paint app

## Image cropping
Why?
- Remove unwanted objects
- Rule of thirds. Separate images into $3 \times 3$ grids, and we place our camera in a way such that the object of interest is on the grid line or its intersection, the picture would look more appealing
- One of the data augmentation techniques

In [ ]:
img = cv.imread("images/dog.jfif")
img_copy = img.copy()

# parameter definition
h, w = img.shape[:2]
n_vertical_grids = 4
n_horizontal_grids = 4

# we need to get the number of pixels for column and row
M = int(h / n_vertical_grids)
N = int(w / n_horizontal_grids)

tiles = []
for y in range(0, h, M):
    for x in range(0, w, N):
        x1 = x + N
        y1 = y + M
        
        if x1 > w and y1 > h:
            x1 = w - 1
            y1 = h - 1
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            
            tile = img[y:h, x:w]
            tiles.append(tile)
        elif y1 > w:
            y1 = h - 1
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            
            tile = img[y:h, x:x1]
            tiles.append(tile)
        elif x1 > w:
            x1 = w - 1
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            
            tile = img[y:y1, x:w]
            tiles.append(tile)
        else:
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            tile = img[y:y1, x:x1]
            tiles.append(tile)
            
show_img("crop", img_copy)

In [ ]:
show_img("patch", tiles[5])

## Exercise
1. Divide the image into 4 equal regions. Swap their positions as shown below: ![dog_swap](image_embed/crop_swap.png)
2. Cover the face of lena with white mask as shown as the following: ![lena_mask](image_embed/lena_mask.png)

Exercise 1

In [ ]:
img = cv.imread("images/dog.jfif")
img_copy = img.copy()

# parameter definition
h, w = img.shape[:2]
n_vertical_grids = 2
n_horizontal_grids = 2

# we need to get the number of pixels for column and row
M = int(h / n_vertical_grids)
N = int(w / n_horizontal_grids)

tiles = []
for y in range(0, h, M):
    for x in range(0, w, N):
        x1 = x + N
        y1 = y + M
        
        if x1 > w and y1 > h:
            x1 = w - 1
            y1 = h - 1
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            
            tile = img[y:h, x:w]
            tiles.append(tile)
        elif y1 > w:
            y1 = h - 1
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            
            tile = img[y:h, x:x1]
            tiles.append(tile)
        elif x1 > w:
            x1 = w - 1
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            
            tile = img[y:y1, x:w]
            tiles.append(tile)
        else:
            cv.rectangle(img_copy, (x, y), (x1, y1), (0, 255, 0), 1)
            tile = img[y:y1, x:x1]
            tiles.append(tile)
        
img_copy[:M, :N] = tiles[3]
img_copy[:M, N:] = tiles[2]
img_copy[M:, :N] = tiles[1]
img_copy[M:, N:] = tiles[0]

show_img("swapped regions", img_copy)

Exercise 2

In [ ]:
img = cv.imread("images/lena.jfif")
if img is None:
    raise Exception("Image not found")
    
h, w = img.shape[:2]
cx, cy = h//2, w//2

cv.rectangle(img, (cx-30, cy-30), (cx+50, cy+50), (255, 255, 255), -1)

show_img("lena mask", img)

## Splitting and merging of color channels

In [ ]:
img = cv.imread("images/lena.jfif")

# split image into separate channels
b, g, r = cv.split(img)

# merge
img_merge = cv.merge((b, g, r))

# test if the two arrays are the same
np.array_equal(img, img_merge)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey=True)
fig.suptitle("Different color channels")

ax1.imshow(b, cmap=plt.cm.gray)
ax1.set(title="blue channel", xticks=[], yticks=[])

ax2.imshow(g, cmap=plt.cm.gray)
ax2.set(title="green channel", xticks=[], yticks=[])

ax3.imshow(r, cmap=plt.cm.gray)
ax3.set(title="red channel", xticks=[], yticks=[])

plt.tight_layout()
plt.show()

In [ ]:
### Apply color filter
img = cv.imread("images/dog.jfif")

colors = ("blue", "green", "red")

channels = cv.split(img)

imgs = []
for i, ch in enumerate(channels):
    img_arr = np.zeros_like(img)
    img_arr[:, :, i] = ch
    imgs.append(img_arr)
    
for img, c in zip(imgs, colors):
    cv.imshow(c, img)
    
cv.waitKey(0)
cv.destroyAllWindows()

## Point operators (mathematical operations)
$$ f_{trans}(\textbf{x}) = \alpha f(\textbf{x}) + \beta $$

In [ ]:
def point_op(img, alpha, beta):
    """Point operators. Arguments:
    1. Source image
    2. multiplier
    3. constant
    """
    img = img.astype(float)
    res = alpha * img + beta
    res = np.clip(res, 0, 255)
    return np.uint8(res)

In [ ]:
# enhance brightness and contrast
img = cv.imread("images/bridge.jfif")

transform = point_op(img, 2, 30)

cv.imshow("original", img)
show_img("transform", transform)

In [ ]:
transform2 = point_op(img, 1, -80)

cv.imshow("original", img)
show_img("transform", transform2)

## gamma correction
$$ O = (\frac{I}{255})^\gamma \times 255 $$

In [ ]:
gamma = 1/2.2

lookUpTable = np.empty((1, 256), dtype=np.uint8)
for i in range(256):
    lookUpTable[0, i] = np.clip(pow(i/255, gamma) * 255, 0, 255)
    
img = cv.imread("images/mountains_prop.jpg")
res = cv.LUT(img, lookUpTable)

cv.namedWindow("original", cv.WINDOW_NORMAL)
cv.imshow("original", img)
show_img("gamma correction", res, adjust=True)

## Images blending (add 2 images together)
get a sense of transparency
$$ g(\textbf{x}) = \alpha f(\textbf{x}) + (1 - \alpha)h(\textbf{x}) + \beta $$

In [ ]:
img = cv.imread("images/lena.jfif")
img2 = cv.imread("images/coins.jfif")

# resize img2
alpha = 0.8
h, w = img.shape[:2]
img2 = cv.resize(img2, (w, h))

# blending
res = cv.addWeighted(img, alpha, img2, 1-alpha, 0)

cv.imshow("img1", img)
cv.imshow("img2", img2)
show_img("blending", res)

## Weekly activity
1. Create a random noise color and grayscale image. You can set your own width and height, but keep the total number of pixels of both images identical.
2. Convert the code chunk found under section Divide an image into smaller patches using cropping into a function with the following signature:

crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color)
* img is the source image
* num_horizontal_grid and num_vertical_grid are the number of patches along x and y axes.
line_color is the color of the grid line.
* The output of the function should be image with grids
3. Display image sequences of smooth transition of two images with different values of α. Refer to code in section "Image blending". Use "lena.jfif" and "coins.jfif" as the base images.
4. Suppose you are a digital content creator and wish to share photo online. However, you wish to protect these images from being stolen or altered by others. Leverage your image processing knowledge to apply watermark on image "travel_hd.jpg". The example of resulting watermarked image are as shown in the following: ![activity_image](images/)

Image courtesy: [Unsplash](#).

Notice the watermark added to the bottom left of the image. You are free to design your own watermark icon.

In [ ]:
import sys
assert sys.version_info >= (3, 7)

import numpy as np
import cv2 as cv
from util_func import show_img

Activity 1

In [ ]:
random_noise = np.random.randint(0, 255, size=(100, 100), dtype=np.uint8)
grayscale = np.zeros((100, 100), dtype=np.uint8) + 128

Activity 2

In [ ]:
def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    """
    * img is the source image
    * num_horizontal_grid and num_vertical_grid are the number of patches along x and y axes.
    * line_color is the color of the grid line. (BGR)
    * The output of the function should be image with grids
    """
    # we need to get the number of pixels for column and row
    M = int(h / num_vertical_grid)
    N = int(w / num_horizontal_grid)

    for y in range(0, h, M):
        for x in range(0, w, N):
            x1 = x + N
            y1 = y + M

            if x1 > w and y1 > h:
                x1 = w - 1
                y1 = h - 1
                cv.rectangle(img, (x, y), (x1, y1), line_color, 1)
            elif y1 > w:
                y1 = h - 1
                cv.rectangle(img, (x, y), (x1, y1), line_color, 1)
            elif x1 > w:
                x1 = w - 1
                cv.rectangle(img, (x, y), (x1, y1), line_color, 1)
            else:
                cv.rectangle(img, (x, y), (x1, y1), line_color, 1)
    return img;

In [ ]:
img = cv.imread("images/dog.jfif")

h, w = img.shape[:2]
n_vertical_grids = 4
n_horizontal_grids = 4

grid_img = crop_grid(img, n_horizontal_grids, n_vertical_grids, (0, 255, 0))
show_img("grid img", grid_img)

Activity 3

In [ ]:
img_lena = cv.imread("images/lena.jfif")
img_coins = cv.imread("images/coins.jfif")

if img_lena is None: 
    raise Exception("lena image not found")
if img_coins is None:
    raise Exception("coin image not found")

h, w = img_lena.shape[:2]
img_coins = cv.resize(img_coins, (w, h))

# transition from coins to lena image
for alpha in np.linspace(0, 1, 11):
    res = cv.addWeighted(img_lena, alpha, img_coins, 1-alpha, 0)
    cv.imshow("blending", res)
    cv.waitKey(100)
    
cv.destroyAllWindows()

Activity 4

In [ ]:
# Using putText to insert 'watermark' into image
img = cv.imread("images/travel_hd.jpg")

if img is None: 
    raise Exception("Image not found")

img = cv.resize(img, None, fx=0.1, fy=0.1)

font = cv.FONT_HERSHEY_SIMPLEX
y, x = img.shape[:2]
res = cv.putText(img, "UCCC2513 watermark", (0, y-50), font, 1, (255, 255, 255), 2)
show_img("watermarked image", res)